In [ ]:
import pandas as pd

### Fontos oszlopok kiválasztása

Az eredeti adathalmaz 19 oszlopa közül kiválasztottuk a számunkra érdekesebbeket.

In [ ]:
year = 2019
month = 1

monthly_data = pd.read_parquet(
    path=f'{year}/{year}-{month:02d}.parquet',
    columns=['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 'trip_distance', 'PULocationID',
             'DOLocationID', 'payment_type', 'fare_amount', 'extra', 'tip_amount', 'total_amount']
)

### Outlierek kiszűrése

Pár oszlopnál tapasztaltunk rendkívül magas/alacsony értékeket (például 2088-as dátum).

Az alábbi szűrési lépéseket tettük meg:

* A dátumoknál meghatároztuk, hogy az adott hónap időkeretében kell lennie a `tpep_pickup_datetime` és a `tpep_dropoff_datetime`-nak.
    * Az is fontos, hogy a `tpep_pickup_datetime` ne legyen előbb mint a `tpep_dropoff_datetime`.
* A `passenger_count` minimum 1 és maximum 6 lehet, mivel ezt írja elő a törvény.
* A `trip_distance` egy pozitív szám, ami kisebb, mint New York átmérője.
* Összesen 263 körzet van, ezért a  `PULocationID` és a `DOLocationID` minimum 1 és maximum 263 lehet.
* A `fare_amount` minimum 2.5\\$.
* Az `extra` 0\\$, 0.5\\$ vagy 1\\$ lehet.
* A `tip_amount` nem lehet negatív, és a `fare_amount` nem lehet nagyobb, mint a `total_amount`.
* Felső korlátnak, csak megadtuk, hogy a 99.99-es percentillisig szűrjön a `fare_amount`, `tip_amount` és a `total_amount` oszlopoknál.

In [ ]:
from calendar import monthrange

filter_columns = ['fare_amount', 'tip_amount', 'total_amount']
percentile = 0.9999

filter_string = ' & '.join(
    (
        f'("{year}, {month}, 1" <= tpep_pickup_datetime)',
        f'(tpep_pickup_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
        f'("{year}, {month}, 1" <= tpep_dropoff_datetime)',
        f'(tpep_dropoff_datetime <= "{year}, {month}, {monthrange(year, month)[1]}")',
        '(tpep_pickup_datetime <= tpep_dropoff_datetime)',
        '(1 <= passenger_count) & (passenger_count <= 6)',
        '(0 < trip_distance) & (trip_distance <= 330)',
        '(1 <= PULocationID) & (PULocationID <= 263)',
        '(1 <= DOLocationID) & (DOLocationID <= 263)',
        '(2.5 <= fare_amount)',
        '(extra in (0, 0.5, 1))',
        '(0 <= tip_amount)',
        '(fare_amount <= total_amount)',
        *(f'({column} <= {column}.quantile({percentile}))' for column in filter_columns)
    )
)

monthly_data = monthly_data.query(filter_string)